In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import layers


In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def create_vgg16_model():
    vgg16 = VGG16(input_shape=(224, 224, 3), weights="imagenet", include_top=False)

    # Freeze all layers in base model
    for layer in vgg16.layers:
        layer.trainable = False

    # Change classifier head with random seed for reproducibility
    x = layers.Flatten()(vgg16.output)
    x = layers.Dense(units=512, activation="relu")(x)
    prediction = layers.Dense(units=7, activation="softmax")(x)
    model = tf.keras.models.Model(inputs=vgg16.input, outputs=prediction)

    return model


In [4]:
# vgg16_model = create_vgg16_model()
from keras.models import load_model

# Load the saved model
vgg16_model = load_model("/content/drive/MyDrive/DataSet/model_checkpoint.h5")

In [5]:

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,  # Randomly rotate images by 30 degrees
    width_shift_range=0.1,  # Randomly shift images horizontally by 10% of the total width
    height_shift_range=0.1,  # Randomly shift images vertically by 10% of the total height
    brightness_range=(0.8, 1.2),  # Randomly adjust brightness between 0.8 to 1.2
    fill_mode='nearest'  # Fill any newly created pixels after rotation or shifting
)


In [ ]:
train_generator = train_datagen.flow_from_directory("/content/drive/MyDrive/DataSet/train/",
    target_size=(224, 224),  # Resize the images to match the input shape of the model
    batch_size=32,
    class_mode='categorical'
)

In [8]:
vgg16_model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
from keras.callbacks import ModelCheckpoint

# Define the checkpoint filepath
checkpoint_filepath = '/content/drive/MyDrive/DataSet/model_checkpoint.h5'

# Create a ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(
    checkpoint_filepath,
    save_weights_only=False,
    save_best_only=False,
    save_freq='epoch',
    verbose=1
)

history = vgg16_model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs according to your needs
    initial_epoch=3,
    callbacks=[checkpoint_callback]
)

Epoch 3/10
773/773 [==============================] - ETA: 0s - loss: 0.5495 - accuracy: 0.8112 
Epoch 3: saving model to /content/drive/MyDrive/DataSet/model_checkpoint.h5
773/773 [==============================] - 14693s 19s/step - loss: 0.5495 - accuracy: 0.8112
Epoch 4/10
511/773 [==================>...........] - ETA: 1:21:32 - loss: 0.5029 - accuracy: 0.8261

In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/test/',  # Path to the directory containing the test images
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
test_loss, test_acc = vgg16_model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_true_labels, y_pred_labels)
print(report)


In [1]:
# Save the model after one epoch
vgg16_model.save("/content/vgg16_model_after_one_epoch.h5")

NameError: ignored